In [1]:
import pandas as pd
import json
import re

In [24]:
run_name = "make_model_union"


# load JSONL
records = []
with open("../aircraft_er_predictions/" + run_name + "_predictions_all.tsv") as f:
    for line in f:
        records.append(json.loads(line))

In [25]:
df = pd.DataFrame(records)
df.head()

,left,right,match,match_confidence
0,COL make VAL CESSNA COL model VAL 180 COL name...,COL make VAL SCBFLG COL model VAL BERGFK COL s...,0,0.999996
1,COL make VAL DOUGLAS COL model VAL DC6 COL nam...,COL code VAL DC4,0,0.999995
2,COL make VAL TAYLORCRAFT COL model VAL BC12 CO...,COL mfr VAL TAYLORCRAFT COL model VAL BC-65,0,0.999859
3,COL make VAL SIKORSKY COL model VAL SH3 COL na...,COL mfr VAL AMD COL model VAL CH-2000 Alarus C...,0,0.999995
4,COL make VAL WACO COL model VAL QDC COL cert V...,COL mfr VAL AEROMOT COL model VAL AMT-200 Supe...,0,0.999994


In [26]:


gold = pd.read_csv("../data/ditto_aircraft/" + run_name + "/all_pairs_with_id.txt")
print(gold.head())

            left_id                    right_id  \
0        CESSNA 180           SCBFLG-BERGFK-III   
1       DOUGLAS DC6                         DC4   
2  TAYLORCRAFT BC12                TCRAFT-BC-65   
3      SIKORSKY SH3         CH2T CH-2000 Alarus   
4          WACO QDC  RF10 AMT-200 Super Ximango   

                                                left  \
0  COL make VAL CESSNA COL model VAL 180 COL name...   
1  COL make VAL DOUGLAS COL model VAL DC6 COL nam...   
2  COL make VAL TAYLORCRAFT COL model VAL BC12 CO...   
3  COL make VAL SIKORSKY COL model VAL SH3 COL na...   
4  COL make VAL WACO COL model VAL QDC COL cert V...   

                                               right  label  
0  COL make VAL SCBFLG COL model VAL BERGFK COL s...      0  
1                                   COL code VAL DC4      0  
2        COL mfr VAL TAYLORCRAFT COL model VAL BC-65      0  
3  COL mfr VAL AMD COL model VAL CH-2000 Alarus C...      0  
4  COL mfr VAL AEROMOT COL model VAL AMT-200 S

In [27]:
df["gold"] = gold["label"]
df["left_id"] = gold["left_id"]
df["right_id"] = gold["right_id"]

In [28]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_true = df["gold"]
y_pred = df["match"]

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification report:\n", classification_report(y_true, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_true, y_pred))

Accuracy: 0.9989489185664823

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    157618
           1       1.00      0.99      1.00     20294

    accuracy                           1.00    177912
   macro avg       1.00      1.00      1.00    177912
weighted avg       1.00      1.00      1.00    177912


Confusion matrix:
 [[157545     73]
 [   114  20180]]


In [29]:
matches = df[(df["gold"] == 1) & (df["match"] == 1)]
matches

,left,right,match,match_confidence,gold,left_id,right_id
17,COL make VAL WACO COL model VAL EGC COL cert V...,COL mfr VAL WACO COL model VAL UKS COL code VA...,1,0.999973,1,WACO EGC,WACC UKS
37,COL make VAL BOEING COL model VAL 757 COL cert...,COL code VAL B753,1,0.999906,1,BOEING 757,B753
41,COL make VAL BELL COL model VAL 205 COL cert V...,COL mfr VAL EAGLE COPTERS COL model VAL 212S E...,1,0.999983,1,BELL 205,UH1 212S Eagle Single
83,COL make VAL MCDONNELL DOUGLAS HELICOPTER COL ...,COL mfr VAL AGUSTA COL model VAL NH-500 COL co...,1,0.999991,1,MCDONNELL DOUGLAS HELICOPTER 500,H500 NH-500
105,COL make VAL CESSNA COL model VAL 208 COL name...,COL mfr VAL CESSNA COL model VAL C-98 COL code...,1,0.999689,1,CESSNA 208,C208 C-98
...,...,...,...,...,...,...,...
177853,COL make VAL BELL COL model VAL 205 COL cert V...,COL mfr VAL AGUSTA COL model VAL Hkp3 COL code...,1,0.999989,1,BELL 205,UH1 Hkp3
177858,COL make VAL BEECH COL model VAL L23F COL name...,COL make VAL BEECH COL model VAL L23F COL seri...,1,0.999969,1,BEECH L23F,1380
177861,COL make VAL BEECH COL model VAL 24 COL name V...,COL mfr VAL BEECH COL model VAL 24 Musketeer S...,1,0.999990,1,BEECH 24,BE24 24 Musketeer Super
177891,COL make VAL TAYLORCRAFT COL model VAL BL COL ...,COL make VAL TCRAFK COL model VAL BL COL serie...,1,0.998928,1,TAYLORCRAFT BL,TCRAFK-BL-65


In [32]:
import re

PAIR_RE = re.compile(r"COL\s+(?P<key>.*?)\s+VAL\s+(?P<val>.*?)(?=\s+COL\s+|$)")

def record_to_kv_list(record: str):
    """Return list like ['make: WACO', 'model: EGC', ...] in original order."""
    s = str(record)
    return [f"{m.group('key').strip()}: {m.group('val').strip()}"
            for m in PAIR_RE.finditer(s)]

In [33]:
# assuming your dataframe is called matches and has columns "left" and "right"
matches["left_kv"]  = matches["left"].apply(record_to_kv_list)
matches["right_kv"] = matches["right"].apply(record_to_kv_list)

In [34]:
matches.head()

,left,right,match,match_confidence,gold,left_id,right_id,left_kv,right_kv
17,COL make VAL WACO COL model VAL EGC COL cert V...,COL mfr VAL WACO COL model VAL UKS COL code VA...,1,0.999973,1,WACO EGC,WACC UKS,"[make: WACO, model: EGC, cert: TC 665, ATC 639]","[mfr: WACO, model: UKS, code: WACC]"
37,COL make VAL BOEING COL model VAL 757 COL cert...,COL code VAL B753,1,0.999906,1,BOEING 757,B753,"[make: BOEING, model: 757, cert: A2NM]",[code: B753]
41,COL make VAL BELL COL model VAL 205 COL cert V...,COL mfr VAL EAGLE COPTERS COL model VAL 212S E...,1,0.999983,1,BELL 205,UH1 212S Eagle Single,"[make: BELL, model: 205, cert: H1SW]","[mfr: EAGLE COPTERS, model: 212S Eagle Single,..."
83,COL make VAL MCDONNELL DOUGLAS HELICOPTER COL ...,COL mfr VAL AGUSTA COL model VAL NH-500 COL co...,1,0.999991,1,MCDONNELL DOUGLAS HELICOPTER 500,H500 NH-500,"[make: MCDONNELL DOUGLAS HELICOPTER, model: 50...","[mfr: AGUSTA, model: NH-500, code: H500]"
105,COL make VAL CESSNA COL model VAL 208 COL name...,COL mfr VAL CESSNA COL model VAL C-98 COL code...,1,0.999689,1,CESSNA 208,C208 C-98,"[make: CESSNA, model: 208, name: CARAVAN, CARG...","[mfr: CESSNA, model: C-98, code: C208]"


In [42]:
matches = matches.sort_values("left_id")

In [50]:
df_agg_cictt = matches.groupby("left_id", as_index=False).agg(count=("left_kv", "count"))

In [55]:
df_agg_cictt = matches.groupby("left_id", as_index=False).agg(
    left_count=("left_kv", "count"),
    right_kv_distinct=("right_kv", lambda s: s.dropna().apply(tuple).nunique()),
)

In [56]:
df_joined = matches.merge(
    df_agg_cictt,
    how="left",
    on=["left_id"],
)

In [57]:
df_joined.head()

,left,right,match,match_confidence,gold,left_id,right_id,left_kv,right_kv,left_count,right_kv_distinct
0,COL make VAL AER PEGASO COL model VAL M100 COL...,COL make VAL AERPEG COL model VAL M100S COL se...,1,0.999951,1,AER PEGASO M100,AERPEG-M100S-M100S,"[make: AER PEGASO, model: M100, cert: NOT CERT...","[make: AERPEG, model: M100S, series: M100S, ce...",3,3
1,COL make VAL AER PEGASO COL model VAL M100 COL...,COL mfr VAL AER-PEGASO COL model VAL M-100S,1,0.999989,1,AER PEGASO M100,AERPEG-M100S-M100S,"[make: AER PEGASO, model: M100, cert: NOT CERT...","[mfr: AER-PEGASO, model: M-100S]",3,3
2,COL make VAL AER PEGASO COL model VAL M100 COL...,COL make VAL AER PEGASO COL model VAL M100 COL...,1,0.999991,1,AER PEGASO M100,25,"[make: AER PEGASO, model: M100, cert: NOT CERT...","[make: AER PEGASO, model: M100, series: S, cer...",3,3
3,COL make VAL AERMACCHI COL model VAL AL60 COL ...,COL make VAL AERMACCHI COL model VAL AL60 COL ...,1,0.999991,1,AERMACCHI AL60,47,"[make: AERMACCHI, model: AL60, name: SANTA MARIA]","[make: AERMACCHI, model: AL60, series: AL60, n...",15,15
4,COL make VAL AERMACCHI COL model VAL AL60 COL ...,COL make VAL AL COL model VAL 60 COL series VA...,1,0.999991,1,AERMACCHI AL60,AL-60-F5,"[make: AERMACCHI, model: AL60, name: SANTA MAR...","[make: AL, model: 60, series: F5, cert: 7A12]",15,15


In [58]:

df_joined.to_csv('cictt_agg_repeat_matches.csv', index=False)